In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet50 import ResNet50
# from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
#from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import os
from PIL import Image
import tensorflow as tf
import tensorflow
from numpy import asarray
import matplotlib.pyplot as plt
tf.compat.v1.enable_eager_execution()
import time
from tensorflow.keras import Model,layers, models, optimizers
from tensorflow.keras.callbacks import  CSVLogger
from tensorflow.keras.constraints import UnitNorm
IMAGE_SIZE = [224, 224]

In [2]:
!nvidia-smi

Tue Apr 11 10:56:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.105.01   Driver Version: 515.105.01   CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   34C    P0    41W / 300W |      0MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   

In [18]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0:2], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        tf.config.experimental.set_memory_growth(gpus[1], True)
        #tf.config.experimental.set_memory_growth(gpus[2], True)
        #tf.config.experimental.set_memory_growth(gpus[3], True)
        #tf.config.experimental.set_memory_growth(gpus[4], True)
        #tf.config.experimental.set_memory_growth(gpus[5], True)
        #tf.config.experimental.set_memory_growth(gpus[6], True)
        #tf.config.experimental.set_memory_growth(gpus[7], True)

        #tf.config.experimental.set_virtual_device_configuration(gpus[3], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=26000)])

        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

#tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10000)])

#tf.config.experimental.set_memory_growth(gpus[0], True)
#tf.config.gpu.set_per_process_memory_fraction(0.75)
#tf.config.gpu.set_per_process_memory_growth(True)

2 Physical GPUs, 2 Logical GPU


In [4]:
# train_path = '/workspace/msp/Retinopathy_dataset/train_vinayak/Opencv_augmented_modified'
# valid_path = '/workspace/msp/Retinopathy_dataset/train_vinayak/Opencv_modified_test'

In [19]:
train_path='/workspace/DP/Tensorflow/tf_comp_dp/Opencv_augmented_modified'
test_path='/workspace/DP/Tensorflow/tf_comp_dp/Opencv_modified_test'

In [20]:
strategy = tf.distribute.MirroredStrategy()
print ('Number of devices: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Number of devices: 2


In [21]:
train_ds= tf.keras.utils.image_dataset_from_directory(
          train_path,
          validation_split=0.2,
          subset="training",
          seed=123,
          image_size=(224, 224),
          batch_size=16)

Found 16656 files belonging to 5 classes.
Using 13325 files for training.


In [22]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(224, 224),
  batch_size=16)

Found 1858 files belonging to 5 classes.
Using 371 files for validation.


In [23]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

In [24]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 1.0


In [25]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


#def cohen_kappa_integerized(y_true, y_pred, num_classes=5, weights=None, metrics_collections=None, updates_collections=None, name=None):
#    kappa, update_op = tensorflow.contrib.metrics.cohen_kappa(
#        y_true, 
#        clip(rint(y_pred), 0, 4), 
#        num_classes, 
#        weights, 
#        metrics_collections, 
#        updates_collections, 
#        name
#     )
#    K.get_session().run(tensorflow.local_variables_initializer())
#    with tensorflow.control_dependencies([update_op]):
#        kappa = tensorflow.identity(kappa)
#    return kappa

In [26]:
with strategy.scope():
       def createModel2(baseModelTrainable):
        image_inputShape = (224, 224, 3)
#         base_model = DenseNet121(
#                     include_top=False, weights='imagenet',
#                     input_shape=image_inputShape, pooling=None
#                 )
        base_model=ResNet50(input_shape=IMAGE_SIZE + [3],weights="imagenet", include_top=False , pooling=None)
        base_model.trainable = baseModelTrainable
        avgPool_layer = tf.keras.layers.GlobalAveragePooling2D() (base_model.output)
        dropOut0 = tf.keras.layers.Dropout(0.5)(avgPool_layer )
        fullLayer1 = tf.keras.layers.Dense(1250,  name='fullLayer1' ) ( dropOut0)
        prelu1 = tf.keras.layers.PReLU(name='pRelu1') (fullLayer1)
        dropOut1 = tf.keras.layers.Dropout(0.5)(prelu1 )
        fullLayer2 = tf.keras.layers.Dense(512,  name='fullLayer2') (dropOut1)
        prelu2 = tf.keras.layers.PReLU( name='pRelu2') (fullLayer2 )
        dropOut2 = tf.keras.layers.Dropout(0.2)(prelu2 )
        out = tf.keras.layers.Dense(5,  name='fullLayer3') (dropOut2 )

        model=models.Model(inputs=base_model.input, outputs=out)

        model.get_layer('pRelu1').set_weights([np.ones(1250)*0.25])
        model.get_layer('pRelu2').set_weights([np.ones(512)*0.25])

        return model


In [27]:
csvFilePath= '/workspace/DP/Tensorflow/Results/2_gpu_DP.csv'
csv_logger = CSVLogger(csvFilePath)

In [28]:
import tensorflow as tf
import tensorflow.keras as tk
import tensorflow_addons as tfa

In [29]:
tfa.metrics.CohenKappa(
    num_classes = 5,
    name = 'cohen_kappa',
    weightage =None,
    sparse_labels = False,
    regression = False,
    dtype = None
)

In [30]:
from tensorflow.keras.optimizers import RMSprop

In [32]:
with strategy.scope():
    model = createModel2(True)
    opt = RMSprop()
    def get_lr_metric(optimizer):
        def lr(y_true, y_pred):
             return optimizer.lr
        return lr

    lr_track = get_lr_metric(opt)

    model.compile(optimizer = opt, 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy',precision_m,f1_m,tfa.metrics.CohenKappa(num_classes=5, sparse_labels=True)])
    model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [33]:
with strategy.scope():
    model = createModel2(True)
#     opti = tf.keras.optimizers.SGD(
#         learning_rate=0.01, momentum=0.9, nesterov=True, name='SGD')
    opti = tf.keras.optimizers.Adagrad(
    learning_rate=0.001,
    initial_accumulator_value=0.01,
    epsilon=1e-06,
    name="Adagrad"
    )
    model.compile(optimizer = opti, 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy',precision_m,f1_m,tfa.metrics.CohenKappa(num_classes=5, sparse_labels=True)])
    model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_4[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [34]:
class TimeHistory(tensorflow.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        #self.times.append(time.time() - self.epoch_time_start)
        self.times.append(time.time() - self.epoch_time_start)
        print(end='\t')
        print("Total epoch training time = ",self.times)
        
#     def on_batch_begin(self, batch, logs=None):
#         self.batch_time_start = time.time()

#     def on_batch_end(self, batch, logs=None):
#         self.batch_time_end.append(time.time() - self.batch_time_start)
#         print("Total batch training time = ",self.batch_time_end)    
        
    

In [35]:
with strategy.scope():
    epochs=50
    time_callback = TimeHistory()
    #model = createModel2(False)
    try:
        history = model.fit(
                            train_ds,
                            validation_data=val_ds, #steps_per_epoch=STEPS_PER_EPOCH,
                            epochs=epochs,
                            callbacks=[csv_logger,time_callback]
                           )
        times = time_callback.times
        
    except Exception as e:
        print(e)

Epoch 1/50


2023-04-11 11:05:39.791573: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_1"
op: "TensorSliceDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_STRING
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 13325
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\025TensorSliceDataset:16"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
        type_id: TFT_TENSOR
        args {
          type_id: TFT_STRING
        }
      }
    }
  }
}



INFO:tensorflow:batch_all_reduce: 220 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 220 all-reduces with algorithm = nccl, num_packs = 1


2023-04-11 11:06:04.616117: I tensorflow/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-04-11 11:06:04.712184: I tensorflow/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600


833/833 [==============================] - ETA: 0s - loss: 1.1050 - accuracy: 0.5465 - precision_m: 0.9348 - f1_m: 1.3604 - cohen_kappa: 0.4318

2023-04-11 11:06:59.248172: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_1"
op: "TensorSliceDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_STRING
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 371
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\025TensorSliceDataset:23"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
        type_id: TFT_TENSOR
        args {
          type_id: TFT_STRING
        }
      }
    }
  }
}



	Total epoch training time =  [85.15624141693115]
833/833 [==============================] - 85s 69ms/step - loss: 1.1050 - accuracy: 0.5465 - precision_m: 0.9348 - f1_m: 1.3604 - cohen_kappa: 0.4318 - val_loss: 0.8038 - val_accuracy: 0.6658 - val_precision_m: 0.8629 - val_f1_m: 1.2621 - val_cohen_kappa: 0.5730
Epoch 2/50
833/833 [==============================] - 48s 57ms/step - loss: 0.7713 - accuracy: 0.6916 - precision_m: 0.9006 - f1_m: 1.3338 - cohen_kappa: 0.6137 - val_loss: 0.6172 - val_accuracy: 0.7466 - val_precision_m: 0.8312 - val_f1_m: 1.2474 - val_cohen_kappa: 0.6780
Epoch 3/50
833/833 [==============================] - 48s 58ms/step - loss: 0.5569 - accuracy: 0.7842 - precision_m: 0.8923 - f1_m: 1.3181 - cohen_kappa: 0.7299 - val_loss: 0.5619 - val_accuracy: 0.7790 - val_precision_m: 0.8405 - val_f1_m: 1.2471 - val_cohen_kappa: 0.7200
Epoch 4/50
833/833 [==============================] - 48s 58ms/step - loss: 0.3520 - accuracy: 0.8695 - precision_m: 0.8763 - f1_m: 1.2856 

In [37]:
# import h5py

In [36]:
model.save('/workspace/DP/Tensorflow/Saved_models/DR_model2.h5')
# tf.keras.models.save_model(
#     '/workspace/DP/Tensorflow/DR_model.h5',
#     overwrite=True,
#     include_optimizer=True,
#     save_format=None,
#     signatures=None,
#     options=None,
#     save_traces=True
# )

In [37]:
import keras

In [38]:
model = keras.models.load_model('/workspace/DP/Tensorflow/Saved_models/DR_model.h5', custom_objects={"precision_m": precision_m, "f1_m":f1_m})

In [ ]:
# with keras.utils.custom_object_scope(custom_objects):
#     new_model = keras.models.clone_model(model)

In [39]:
#model = tf.keras.models.load_model('/workspace/DP/Tensorflow/Saved_models/DR_model.h5')
# model=keras.models.load_model('/workspace/DP/Tensorflow/Saved_models/DR_model.h5')
# model

img = Image.open("/workspace/DP/Tensorflow/tf_comp_dp/Opencv_augmented_modified/4/0_f_r_-1_16ce555748d8.png")
img = img.resize((224,224))
img = np.array(img)
img = img/255.0
img = np.expand_dims(img, axis=0)

In [40]:
pred = model.predict(img)
pred = np.argmax(pred)
print(pred)

1/1 [==============================] - 1s 970ms/step
1


In [41]:
http://localhost:8886/view/Tensorflow/tf_comp_dp/Opencv_augmented_modified/3/0_f_r_-1_042470a92154.png
        http://localhost:8886/view/Tensorflow/tf_comp_dp/Opencv_augmented_modified/4/0_f_r_-1_10fca1abf338.png
                http://localhost:8886/view/Tensorflow/tf_comp_dp/Opencv_augmented_modified/0/f_r_-1002c21358ce6.png

SyntaxError: invalid syntax (3920391320.py, line 1)